In [1]:
import requests
from bs4 import BeautifulSoup
import pandas
import os
import pdfkit
import imgkit
import random
import time

In [2]:
proxy = [{"http":"103.37.141.69:80"},
{"http":"14.215.212.37:9168"},
{"http":"111.59.199.58:8188"},
{"http":"14.215.212.37:9168"},
{"http":"223.82.60.202:8060"},
{"http":"222.78.6.190:8083"},
{"http":"106.15.197.250:8001"},
{"http":"222.66.202.6:80"},
{"http":"101.34.214.152:8001"},
{"http":"14.215.212.37:9168"},
{"http":"202.55.5.209:8090"},
{"http":"111.3.111.179:30001"},
{"http":"124.93.201.59:42672"},
{"http":"106.15.197.250:8001"},
{"http":"111.59.199.58:8188"},
{"http":"106.15.197.250:8001"},
{"http":"115.218.5.177:9000"},
{"http":"117.114.149.66:55443"},
{"http":"47.92.234.75:80"},
{"http":"112.14.47.6:52024"}]

headers={'User-agent': 'Mozilla/5.0'}

In [3]:
def findElitePostPageNumber(groupID):
    startURL = "https://www.douban.com/group/" + str(groupID) + "/discussion?start=0&type=elite"
    r = requests.get(startURL, headers=headers, proxies=random.choice(proxy))
    c = r.content
    soup = BeautifulSoup(c, "html.parser")
    pageNumber = int(soup.find("span", {"class":"thispage"}).attrs["data-total-page"])
    return pageNumber

In [4]:
def findElitePostURLs(groupID):
    ElitePostURLs = []
    baseURL = "https://www.douban.com/group/" + str(groupID) + "/discussion?start="
    pageNumber = findElitePostPageNumber(groupID)
    for page in range(0, pageNumber*25, 25):
        currentPageURL = baseURL + str(page) + "&type=elite"
        # check process
        print(page)
        r = requests.get(currentPageURL, headers=headers, proxies=random.choice(proxy))
        c = r.content
        soup = BeautifulSoup(c, "html.parser")
        all = soup.find_all("td", {"class":"title"})
        for item in all:
            postInfo = {}
            # find post url
            elitePostURL = item.find("a").attrs["href"]
            postInfo["postURL"] = elitePostURL
            # find post title                                 
            postInfo["postTitle"] = item.find("a").attrs["title"].replace(' ', '-').replace('/', '')

            ElitePostURLs.append(postInfo)
            
    df = pandas.DataFrame(ElitePostURLs)
    df = df.drop_duplicates("postURL")
    df = df.reset_index(drop=True)
    return df

In [5]:
basePath = '/Users/sw/Desktop/DoubanSpider'
def createOutputDir(groupID):
    groupPath = os.path.join(basePath, "Group"+str(groupID))
    os.mkdir(groupPath)
    groupURLs = findElitePostURLs(groupID)
    for i in range(len(groupURLs)):
        postTitle = groupURLs["postTitle"][i]
        postPath = os.path.join(groupPath, postTitle)
        os.mkdir(postPath)

In [6]:
def savePostInsideLinks(groupID, postURL, postTitle):
    r = requests.get(postURL, headers=headers, proxies=random.choice(proxy))
    c = r.content
    soup = BeautifulSoup(c, "html.parser")

    links = soup.find_all("a", {"class":"link"})
    # 确认有没有链接
    if len(links):
        postInsideLinks = []
        for element in links:
            insideLinkInfo = {}
            insideLinkTitle = element.text
            insideLinkInfo["linkTitle"] = insideLinkTitle
            insidelink = element.attrs["href"]
            insideLinkInfo["linkAddress"] = insidelink
            postInsideLinks.append(insideLinkInfo)
        postInsideLinks = pandas.DataFrame(postInsideLinks)
    
        postPath = basePath + "/Group"+str(groupID)+"/" + postTitle+"/"

        # csv format
        postInsideLinks.to_csv(postPath+"link"+".csv", encoding="utf-8")
        # txt format
        postInsideLinks.to_csv(postPath+"link"+".txt", sep='\t', index=False, encoding="utf-8")
            

In [7]:
def postImageGifDownloader(groupID, postURL, postTitle):
    postPath = basePath + "/Group"+str(groupID)+"/" + postTitle+"/"

    r = requests.get(postURL, headers=headers, proxies=random.choice(proxy))
    c = r.content
    soup = BeautifulSoup(c, "html.parser")

    images = soup.find_all("div", {"class":"image-wrapper"})
    # check whether exist images in the post
    if len(images):
        for i in range(len(images)):
            try:
                # is a gif
                imageLink = images[i].find("img").attrs["data-original-url"]
                imageName = '{}-{}.gif'.format(postTitle, i)
            except:
                # is a jpg
                imageLink = images[i].find("img").attrs["src"]
                imageName = '{}-{}.jpg'.format(postTitle, i)    
            with open(postPath+imageName, "wb") as f:
                postImage = requests.get(imageLink, headers=headers, proxies=random.choice(proxy)).content
                f.write(postImage)
                print('Writing', imageName)

In [1]:
def savePostContentText(groupID, postURL, postTitle):
    r = requests.get(postURL, headers=headers, proxies=random.choice(proxy))
    c = r.content
    soup = BeautifulSoup(c, "html.parser")

    contentTexts = soup.find_all("p", class_=False, id=False, alignment="")
    result = []
    for line in contentTexts:
        result.append(line.text.replace("\n", ""))
    result = pandas.DataFrame(result)

    basePath = '/Users/sw/Desktop/DoubanSpider'
    postPath = basePath + "/Group"+str(groupID)+"/" + postTitle+"/"
    result.to_csv(postPath+"正文"+".txt", sep='\t',index=False, encoding="utf-8")

In [10]:
def savePostComments(groupID, postURL, postTitle):
    r = requests.get(postURL, headers=headers, proxies=random.choice(proxy))
    c = r.content
    soup = BeautifulSoup(c, "html.parser")
    
    comments = soup.find_all("div", {"class":"reply-doc content"})
    postComments = []
    for i in range(len(comments)):
        commentInfo = {}
        commentUser = comments[i].find("a").text
        commentContent = comments[i].find("p").text
        commentInfo["user"] = commentUser
        commentInfo["content"] = commentContent
        postComments.append(commentInfo)
    
    postComments = pandas.DataFrame(postComments)

    postPath = basePath + "/Group"+str(groupID)+"/" + postTitle+"/"
    # csv format
    postComments.to_csv(postPath+"comment"+".csv", encoding="utf-8")
    # txt format
    postComments.to_csv(postPath+"comment"+".txt",sep='\t',index=False, encoding="utf-8")

In [11]:
def savePostAsPdf(groupID, postURL, postTitle):
    postPath = basePath + "/Group"+str(groupID)+"/" + postTitle+"/"
    pdfoptions = {'dpi':400}
    pdfkit.from_url(postURL, postPath+postTitle+"-帖子全部"+".pdf", options=pdfoptions)

In [ ]:
def groupElitePostCollector(groupID):
    # 找到豆瓣小组所有精华帖子的URL和标题名
    groupElitePostInfos = findElitePostURLs(groupID)

    # 创建路径 方便数据整理
    createOutputDir(groupID)
    for i in range(len(groupElitePostInfos)):
        postURL = groupElitePostInfos["postURL"][i]
        postTitle = groupElitePostInfos["postTitle"][i]

        # 1. 储存精华帖中的链接和链接标题到 csv txt
        savePostInsideLinks(groupID, postURL, postTitle)

        # 2. 储存精华帖中的所有图片或者动图，按照出现顺序标号命名
        postImageGifDownloader(groupID, postURL, postTitle)

        # 3. 储存精华帖中的所有正文信息
        savePostContentText(groupID, postURL, postTitle)

        # 4. 储存精华帖中所有评论内容 评论ID 评论内容 到csv txt
        savePostComments(groupID, postURL, postTitle)

        # 5. 把精华帖生成pdf文件到指定路径
        savePostAsPdf(groupID, postURL, postTitle)